In [2]:
import pgrtk
pgrtk.__version__

# Load index
sdb = pgrtk.SeqIndexDB()
sdb.load_from_agc_index("/data/data/pgr-tk-HGRP-y1-evaluation-set-v0")

In [5]:
def load_sub_seq(loci:tuple, compliment=False, database=sdb) -> list:
    """ Reverse complement a sequence as a list of bytes (unsigned 8bit interger).

    Parameters
    ----------
    database    : loaded in .mdb object
    loci        : tuple containing ('genome_file', 'chromosome', start, end)
    compliment  : bool indicating whether the sequence should be complimented or not
    Returns
    -------
    list of bytes 
        the list of bytes of the given subsequence

    """
    subseq = database.get_sub_seq(loci[0], loci[1], loci[2], loci[3])
    if compliment: subseq = pgrtk.rc_byte_seq(subseq)
    return subseq

def get_seq_list(ROI_seq: list, sdb=sdb, padding: int=1.5e5, plot=True) -> list:
    '''
    Get a list of sequences from sdb index that match the ROI_seq.
    :param sdb: SeqIndexDB object, the PGR index
    :param ROI_seq: ROI sequence
    :param padding: padding length  
    :param plot:  
    '''

    ROI_len = len(ROI_seq)

    # query the PGR index to find matches of this ROI
    query_results = pgrtk.query_sdb(sdb, ROI_seq)
    seq_list = []
    seq_info = sdb.seq_info.copy() # Cache seq_info to reduce Rust HashMap to Python dictionary conversion in a loop
    i = 1
    if plot:
        plt.figure(figsize=(36, 36))
    for k in list(query_results.keys()):
        ctg_name, source, _ = seq_info[k]
        seq_id = k
        rgns = query_results[k].copy()
        # rgns = pgrtk.merge_regions(rgns,tol=1000) # if additional region merge needed
        for rgn in rgns:
            b, e, length, orientation, aln = rgn #beginning, end, length, orientation, alignments
            aln.sort()
            # Filters out reads that are not the entire length of the gene.
            #if aln[0][0][0] > padding or aln[-1][0][1] < padding + ROI_len: 
            #    continue
            
            seq =  sdb.get_sub_seq(source, ctg_name, b, e)
            # print(source, ctg_name, b, e)
            if orientation == 1:
                seq = pgrtk.rc_byte_seq(seq)

            seq_list.append(("{}_{}_{}_{}".format(ctg_name, b, e, orientation), seq))
            
            if e-b < len(ROI_seq) * 0.50: # ignore partial match
                continue
            else:
                x, y = pgrtk.get_shmmr_dots(ROI_seq, seq, 32, 32, 1, 1)
                if i <= 25 and plot:
                    plt.subplot(5,5,i)
                    plt.plot(x, y, ".", markersize=0.1)
                    ylabel = "#".join(ctg_name.split("#")[:2])
                    if len(ylabel) > 20:
                        ylabel = ylabel[:20]
                    plt.ylabel(ylabel)
                    # plt.plot([padding, padding],[0, max(y)], "r")
                    # plt.plot([padding+ROI_len, padding+ROI_len],[0, max(y)], "r")
                i += 1

    return seq_list

In [29]:
sdb.seq_info

{4774: ('HG00733#1#JAHEPQ010000084.1',
  'HG00733.paternal.f1_assembly_v2_genbank.fa',
  18668836),
 22376: ('HG02572#2#JAHAOV010000598.1',
  'HG02572.maternal.f1_assembly_v2_genbank.fa',
  28338),
 17858: ('HG02145#2#JAHKSF010000056.1',
  'HG02145.maternal.f1_assembly_v2_genbank.fa',
  22433156),
 4203: ('HG00733#2#JAHEPP010000122.1',
  'HG00733.maternal.f1_assembly_v2_genbank.fa',
  2364314),
 9458: ('HG01175#2#JAHALZ010000188.1',
  'HG01175.maternal.f1_assembly_v2_genbank.fa',
  365093),
 17057: ('HG02109#2#JAHEPF010000181.1',
  'HG02109.maternal.f1_assembly_v2_genbank.fa',
  134160),
 11835: ('HG01358#1#JAGYZB010000100.1',
  'HG01358.paternal.f1_assembly_v2_genbank.fa',
  2536849),
 21573: ('HG02559#1#JAGYVK010000055.1',
  'HG02559.paternal.f1_assembly_v2_genbank.fa',
  256256),
 33876: ('HG03540#1#JAGYVY010000473.1',
  'HG03540.paternal.f1_assembly_v2_genbank.fa',
  29389),
 38701: ('chr19_KI270891v1_alt_hg38', 'hg38_tagged.fa', 170680),
 13678: ('HG01928#2#JAGYVP010000161.1',
  '

In [20]:
# Assign Subsequences
# (Genome File Name, Chromosome Name, Seq Start Location, Seq End Location)
C4 = ('hg38_tagged.fa', "chr6_hg38", 31835263, 31855887)

# Get seqs
C4_seq = load_sub_seq(C4)

# Get seq list
C4_list = get_seq_list(C4_seq, plot=False)


In [6]:
ROI_lists = [C4_list, HTT_list]

3083408